<a href="https://colab.research.google.com/github/mburkey3/Intro_to_ML/blob/main/Homework5/Homework5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file_path = '/content/drive/My Drive/Intro_to_ML/Datasets/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
# Binary map Outputs to 0 or 1
# M = 1
# B = 0
# Map function
def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
dataset[varlist] = dataset[varlist].apply(binary_map)
# Remove uneeded information
dataset.pop('furnishingstatus')
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
0,13300000,7420,4,2,3,1,0,0,0,1,2,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0


In [5]:
# Standarize the data to remove size differences
stdScaler = StandardScaler()
num_vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea']
dataset[num_vars] = stdScaler.fit_transform(dataset[num_vars])
# Seperate data into inputs and outputs

Y = dataset.pop('price')
X = dataset.values
# convert to tensors
X = torch.tensor(X)
Y = torch.tensor(Y)
X, Y

(tensor([[ 1.0467,  1.4034,  1.4218,  ...,  1.4726,  1.5177,  1.8049],
         [ 1.7570,  1.4034,  5.4058,  ...,  1.4726,  2.6794, -0.5540],
         [ 2.2182,  0.0473,  1.4218,  ..., -0.6791,  1.5177,  1.8049],
         ...,
         [-0.7059, -1.3089, -0.5702,  ..., -0.6791, -0.8057, -0.5540],
         [-1.0334,  0.0473, -0.5702,  ..., -0.6791, -0.8057, -0.5540],
         [-0.5998,  0.0473, -0.5702,  ..., -0.6791, -0.8057, -0.5540]],
        dtype=torch.float64),
 tensor([ 4.5664e+00,  4.0045e+00,  4.0045e+00,  3.9858e+00,  3.5550e+00,
          3.2553e+00,  2.8807e+00,  2.8807e+00,  2.7309e+00,  2.6934e+00,
          2.6934e+00,  2.6297e+00,  2.4312e+00,  2.3938e+00,  2.3938e+00,
          2.3188e+00,  2.3188e+00,  2.2439e+00,  2.2065e+00,  2.1877e+00,
          2.1315e+00,  2.0941e+00,  2.0754e+00,  2.0754e+00,  2.0379e+00,
          2.0192e+00,  1.9780e+00,  1.9443e+00,  1.9443e+00,  1.9443e+00,
          1.9443e+00,  1.9443e+00,  1.8881e+00,  1.8319e+00,  1.7944e+00,
          1

In [6]:
# Split data into training and validation sets
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 99, 529, 435,  74, 164,  49, 265, 507, 410, 538, 161, 325, 321, 191,
         533, 470, 473,  50, 302, 367, 485,  65, 342, 364, 349, 127, 354,   8,
         520, 344, 296, 255, 256,   0, 108,  47, 257, 290, 483, 406,   7, 235,
         397,  63, 258,  14, 386, 355, 273, 468,  70, 527, 381, 308, 453, 329,
         243, 225,   1, 524,  93, 320, 238, 224, 496, 412, 445, 452, 356, 535,
         351,  55, 420, 298,  29, 217,   9, 369, 181, 536, 152, 160, 454,  68,
         111,  94,  53, 261, 337, 233, 140, 162, 145, 312, 113, 268, 303, 404,
         315, 295, 134,  73, 487, 177, 490, 359, 423, 289, 521, 534, 185, 291,
         133, 539, 350, 541, 220,   3,  41, 411, 540, 254, 112, 525, 244, 196,
         352, 338, 481,  22, 142, 219, 179,  78,  40, 448, 391, 283, 260,  10,
         149,  86, 241, 234, 407, 218, 474,  38, 125, 285, 173, 523, 387, 278,
         528, 252, 330, 363, 388, 212, 154, 542, 389,  24,  12, 231, 221, 206,
          95, 250, 447,  67, 193, 396, 146, 413, 208

In [7]:
# Training set
train_X = X[train_indices]
train_Y = Y[train_indices]
# Validation set
val_X = X[val_indices]
val_Y = Y[val_indices]

In [8]:
# Linear model
#weights = ['w1', 'w2', 'w3', 'w4', 'w5']
def model(x, w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11) :
    return w0 + x[:,0]*w1 + x[:,1]*w2 + x[:,2]*w3 + x[:,3]*w4 + x[:,4]*w5 + x[:,5]*w6 + x[:,6]*w7 + x[:,7]*w8 + x[:,8]*w9 + x[:,9]*w10 + x[:,10]*w11

# Loss function
def loss_fn(y_p, y) :
    squared_diffs = (y_p - y)**2
    return squared_diffs.mean()

In [9]:
# Training Function
def training_fn(epochs, opt, params, x_train, y_train, x_val, y_val) :
    for epoch in range(1, epochs + 1):
        train_p = model(x_train, *params)
        train_loss = loss_fn(train_p, y_train)

        val_p = model(x_val, *params)
        val_loss = loss_fn(val_p, y_val)

        opt.zero_grad()
        train_loss.backward()
        opt.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

    return params

In [10]:
# Training: SGD
training_lr = [0.1, 0.01, 0.001, 0.0001]
for n in training_lr :
    parameters = torch.tensor([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], requires_grad=True)
    optim_SGD = optim.SGD([parameters], lr=n)
    print("Learning Rate:", n)
    training_fn(
        epochs = 5000,
        opt = optim_SGD,
        params = parameters,
        x_train = train_X,
        y_train = train_Y,
        x_val = val_X,
        y_val = val_Y
    )
    print(" ")

Learning Rate: 0.1
Epoch 500, Training loss 0.2883, Validation loss 0.4947
Epoch 1000, Training loss 0.2883, Validation loss 0.4947
Epoch 1500, Training loss 0.2883, Validation loss 0.4947
Epoch 2000, Training loss 0.2883, Validation loss 0.4947
Epoch 2500, Training loss 0.2883, Validation loss 0.4947
Epoch 3000, Training loss 0.2883, Validation loss 0.4947
Epoch 3500, Training loss 0.2883, Validation loss 0.4947
Epoch 4000, Training loss 0.2883, Validation loss 0.4947
Epoch 4500, Training loss 0.2883, Validation loss 0.4947
Epoch 5000, Training loss 0.2883, Validation loss 0.4947
 
Learning Rate: 0.01
Epoch 500, Training loss 0.2883, Validation loss 0.4947
Epoch 1000, Training loss 0.2883, Validation loss 0.4947
Epoch 1500, Training loss 0.2883, Validation loss 0.4947
Epoch 2000, Training loss 0.2883, Validation loss 0.4947
Epoch 2500, Training loss 0.2883, Validation loss 0.4947
Epoch 3000, Training loss 0.2883, Validation loss 0.4947
Epoch 3500, Training loss 0.2883, Validation loss

In [11]:
# Training: ADAM
# Reset parameters
for n in training_lr:
    parameters = torch.tensor([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], requires_grad=True)
    optim_ADAM = optim.Adam([parameters], lr=n)
    print("Learning Rate:", n)
    training_fn(
        epochs = 5000,
        opt = optim_ADAM,
        params = parameters,
        x_train = train_X,
        y_train = train_Y,
        x_val = val_X,
        y_val = val_Y
    )
    print(" ")

Learning Rate: 0.1
Epoch 500, Training loss 0.2883, Validation loss 0.4947
Epoch 1000, Training loss 0.2883, Validation loss 0.4947
Epoch 1500, Training loss 0.2883, Validation loss 0.4952
Epoch 2000, Training loss 0.2883, Validation loss 0.4947
Epoch 2500, Training loss 0.2883, Validation loss 0.4945
Epoch 3000, Training loss 0.2883, Validation loss 0.4937
Epoch 3500, Training loss 0.2883, Validation loss 0.4947
Epoch 4000, Training loss 0.2883, Validation loss 0.4948
Epoch 4500, Training loss 0.2883, Validation loss 0.4947
Epoch 5000, Training loss 0.2883, Validation loss 0.4949
 
Learning Rate: 0.01
Epoch 500, Training loss 0.2883, Validation loss 0.4947
Epoch 1000, Training loss 0.2883, Validation loss 0.4947
Epoch 1500, Training loss 0.2883, Validation loss 0.4947
Epoch 2000, Training loss 0.2883, Validation loss 0.4947
Epoch 2500, Training loss 0.2883, Validation loss 0.4947
Epoch 3000, Training loss 0.2883, Validation loss 0.4947
Epoch 3500, Training loss 0.2883, Validation loss